## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

# \[Capacity Queries\]

> - Database Sizes Q1)
> - Capture Database Sizes to a table (Q2)
> - Filesize growth rates (Q3)

> - Database Sizes Q1)

In [ ]:
/* 01 Data sizes.sql  #SQL */
--============================================================================================
--01 Database and log sizes per database and freespace 
--============================================================================================
IF (
		SELECT OBJECT_ID('Tempdb.dbo.#DatabaseSizes')
		) IS NOT NULL
	DROP TABLE #DatabaseSizes
GO

CREATE TABLE #DatabaseSizes (
	SrvName SYSNAME
	,DatabaseName SYSNAME
	,FileName SYSNAME
	,PhysicalName SYSNAME
	,TotalSizeinMB DECIMAL(12, 2)
	,AvailableSpaceInMB DECIMAL(12, 2)
	,file_id INT
	,FilegroupName SYSNAME NULL
	,Type_desc VARCHAR(10)
	)

EXEC sp_MSForEachDB 'USE [?] INSERT INTO #DatabaseSizes
SELECT @@Servername AS SrvName,
db_name() AS DatabaseName,
f.name AS [File Name] , f.physical_name AS [Physical Name], 
CAST((f.size/128.0) AS DECIMAL(15,2)) AS [Total Size in MB],
CAST(f.size/128.0 - CAST(FILEPROPERTY(f.name, ''SpaceUsed'') AS int)/128.0 AS DECIMAL(15,2)) 
AS [Available Space In MB], [file_id], fg.name AS [Filegroup Name],f.Type_desc
FROM sys.database_files AS f WITH (NOLOCK) 
LEFT OUTER JOIN sys.data_spaces AS fg WITH (NOLOCK) 
ON f.data_space_id = fg.data_space_id OPTION (RECOMPILE);
'

SELECT @@servername AS SrvName
	,DatabaseName
	,SUM(TotalSizeinMB) AS [Total Size in MB]
	,SUM(AvailableSpaceInMB) AS [Available Space In MB]
	,Type_desc
FROM #DatabaseSizes
GROUP BY SrvName
	,DatabaseName
	,Type_desc

SELECT @@SERVERNAME AS SrvName
	,DatabaseName
	,GETDATE() AS ReportDate
	,SUM(CASE 
			WHEN Type_desc = 'ROWS'
				THEN TotalSizeinMB
			ELSE 0
			END) AS DataSizeTotal
	,SUM(CASE 
			WHEN Type_desc = 'ROWS'
				THEN AvailableSpaceInMB
			ELSE 0
			END) AS DataSizefree
	,SUM(CASE 
			WHEN Type_desc = 'LOG'
				THEN TotalSizeinMB
			ELSE 0
			END) AS LogSizeTotal
	,SUM(CASE 
			WHEN Type_desc = 'LOG'
				THEN AvailableSpaceInMB
			ELSE 0
			END) AS LogSizefree
	,SUM(TotalSizeinMB) AS TotalSizeinMB
FROM #DatabaseSizes
GROUP BY SrvName
	,DatabaseName



> - Capture Database Sizes to a table (Q2)

In [ ]:
--===================================================================================
--02 Capture size information
--===================================================================================

/*
--uncomment if requirement to store capacity infomation
IF (
		SELECT OBJECT_ID('DatabaseSizes')
		) IS NOT NULL
	DROP TABLE DatabaseSizes
GO

CREATE TABLE DatabaseSizes (
	SrvName SYSNAME
	,DatabaseName SYSNAME
	,ReportDate DATETIME
	,DataSizeTotal DECIMAL(12, 2)
	,DataSizefree DECIMAL(12, 2)
	,LogSizeTotal DECIMAL(12, 2)
	,LogSizefree DECIMAL(12, 2)
	,TotalSizeinMB DECIMAL(12, 2)
	,
	)
*/
IF (
		SELECT OBJECT_ID('Tempdb.dbo.#DatabaseSizes')
		) IS NOT NULL
	DROP TABLE #DatabaseSizes
GO

CREATE TABLE #DatabaseSizes (
	SrvName SYSNAME
	,DatabaseName SYSNAME
	,FileName SYSNAME
	,PhysicalName SYSNAME
	,TotalSizeinMB DECIMAL(12, 2)
	,AvailableSpaceInMB DECIMAL(12, 2)
	,file_id INT
	,FilegroupName SYSNAME NULL
	,Type_desc VARCHAR(10)
	)

EXEC sp_MSForEachDB 'USE [?] INSERT INTO #DatabaseSizes
SELECT @@Servername AS SrvName,
db_name() AS DatabaseName,
f.name AS [File Name] , f.physical_name AS [Physical Name], 
CAST((f.size/128.0) AS DECIMAL(15,2)) AS [Total Size in MB],
CAST(f.size/128.0 - CAST(FILEPROPERTY(f.name, ''SpaceUsed'') AS int)/128.0 AS DECIMAL(15,2)) 
AS [Available Space In MB], [file_id], fg.name AS [Filegroup Name],f.Type_desc
FROM sys.database_files AS f WITH (NOLOCK) 
LEFT OUTER JOIN sys.data_spaces AS fg WITH (NOLOCK) 
ON f.data_space_id = fg.data_space_id OPTION (RECOMPILE);
'

SELECT *
FROM #DatabaseSizes

SELECT SrvName
	,DatabaseName
	,SUM(TotalSizeinMB) AS [Total Size in MB]
	,SUM(AvailableSpaceInMB) AS [Available Space In MB]
	,Type_desc
FROM #DatabaseSizes
GROUP BY SrvName
	,DatabaseName
	,Type_desc



SELECT SrvName
	,DatabaseName
	,GETDATE() AS ReportDate
	,SUM(CASE 
			WHEN Type_desc = 'ROWS'
				THEN TotalSizeinMB
			ELSE 0
			END) AS DataSizeTotal
	,SUM(CASE 
			WHEN Type_desc = 'ROWS'
				THEN AvailableSpaceInMB
			ELSE 0
			END) AS DataSizefree
	,SUM(CASE 
			WHEN Type_desc = 'LOG'
				THEN TotalSizeinMB
			ELSE 0
			END) AS LogSizeTotal
	,SUM(CASE 
			WHEN Type_desc = 'LOG'
				THEN AvailableSpaceInMB
			ELSE 0
			END) AS LogSizefree
	,SUM(TotalSizeinMB) AS TotalSizeinMB
FROM #DatabaseSizes
GROUP BY SrvName
	,DatabaseName


> - Filesize growth rates (Q3)

In [ ]:
--===================================================================================
--03 File sizes growth information
--===================================================================================
SELECT @@SERVERNAME AS Servername
	,'Database Name' = DB_NAME(database_id)
	,'FileName' = NAME
	,FILE_ID
	,'size' = CONVERT(NVARCHAR(15), CONVERT(BIGINT, size) * 8) + N' KB'
	,'maxsize' = (
		CASE max_size
			WHEN - 1
				THEN N'Unlimited'
			ELSE CONVERT(NVARCHAR(15), CONVERT(BIGINT, max_size) * 8) + N' KB'
			END
		)
	,'growth' = (
		CASE is_percent_growth
			WHEN 1
				THEN CONVERT(NVARCHAR(15), growth) + N'%'
			ELSE CONVERT(NVARCHAR(15), CONVERT(BIGINT, growth) * 8) + N' KB'
			END
		)
	,'type_desc' = type_desc
	,physical_name
FROM sys.master_files
ORDER BY database_id


## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)